<a href="https://colab.research.google.com/github/alzoqm/jp2ko_translation_model/blob/main/transformer_translation_kojp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈 및 vocab 로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [38]:
import sentencepiece as spm
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Layer
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import urllib.request
import time
import os
import tqdm

from tqdm import *

In [4]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.87.130.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.87.130.178:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# 모델

In [5]:
class PositionalEncoding(Layer):
  def __init__(self, position, d_model):
    super().__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(1000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model = d_model
    )
    sines = tf.math.sin(angle_rads[:, 0::2])
    cos = tf.math.cos(angle_rads[:, 1::2])
    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cos

    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


In [6]:
class MultiHeadAttention(Layer):
  def __init__(self, d_model, num_heads):
    super().__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model // num_heads
    self.depth = d_model // num_heads
    self.scale = 1 / (self.depth ** 0.5)

    self.q_linear = tf.keras.layers.Dense(d_model)
    self.k_linear = tf.keras.layers.Dense(d_model)
    self.v_linear = tf.keras.layers.Dense(d_model)

    self.linear = tf.keras.layers.Dense(d_model)

  def call(self, q, k, v, mask):
    #batch_size = q.shape[0]
    batch_size = tf.shape(q)[0]

    q = self.q_linear(q)
    k = self.k_linear(k)
    v = self.v_linear(v)

    q = tf.reshape(q, shape=(batch_size, -1, self.num_heads, self.depth))
    q = tf.transpose(q, perm=[0, 2, 1, 3])

    k = tf.reshape(k, shape=(batch_size, -1, self.num_heads, self.depth))
    k = tf.transpose(k, perm=[0, 2, 1, 3])

    v = tf.reshape(v, shape=(batch_size, -1, self.num_heads, self.depth))
    v = tf.transpose(v, perm=[0, 2, 1, 3])

    matmul_qk = tf.matmul(q, k, transpose_b=True)
    logits = matmul_qk / self.scale

    if mask is not None:
      logits += (mask * -1e9)

    attn_mat = tf.nn.softmax(logits, axis=-1)
    outputs = tf.matmul(attn_mat, v)
    outputs = tf.transpose(outputs, perm=[0, 2, 1, 3])

    outputs = tf.reshape(outputs, shape=(batch_size, -1, self.d_model))
    outputs = self.linear(outputs)
    return outputs, attn_mat

In [7]:
class MLP(Layer):
  def __init__(self, dff, d_model):
    super().__init__()
    self.linear1 = tf.keras.layers.Dense(dff)
    self.linear2 = tf.keras.layers.Dense(d_model)

  def call(self, inputs):
    outputs = self.linear1(inputs)
    outputs = tf.nn.gelu(outputs)
    outputs = self.linear2(outputs)
    return outputs

In [8]:
class EncoderLayer(tf.keras.Model):
  def __init__(self, dff, d_model, num_heads, dropout):
    super().__init__()
    self.attn_layer = MultiHeadAttention(d_model, num_heads)
    self.mlp = MLP(dff, d_model)

    self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-9)
    self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-9)

    self.dropout1 = tf.keras.layers.Dropout(dropout)
    self.dropout2 = tf.keras.layers.Dropout(dropout)

  def call(self, inputs, mask):
    attn_outputs, attn_mat = self.attn_layer(inputs, inputs, inputs, mask)
    attn_outputs = self.dropout1(attn_outputs)
    attn_outputs = self.layer_norm1(inputs + attn_outputs)

    outputs = self.mlp(attn_outputs)
    outputs = self.dropout2(outputs)
    outputs = self.layer_norm2(attn_outputs + outputs)
    return outputs, attn_mat

In [9]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, num_layers, dff, d_model, num_heads, dropout):
    super().__init__()
    self.word_embedding = tf.keras.layers.Embedding(vocab_size, d_model)
    self.pos_embedding = PositionalEncoding(vocab_size, d_model)

    self.dropout = tf.keras.layers.Dropout(dropout)

    self.encoder_layer = [EncoderLayer(dff, d_model, num_heads, dropout) for _ in range(num_layers)]

  def call(self, inputs, mask):
    attn_mats = []
    embedding = self.word_embedding(inputs)
    embedding = self.pos_embedding(embedding)
    outputs = self.dropout(embedding)

    for layer in self.encoder_layer:
      outputs, attn_mat = layer(outputs, mask)
      attn_mats.append(attn_mat)
    
    return outputs, attn_mats

In [10]:
class DecoderLayer(tf.keras.Model):
  def __init__(self, dff, d_model, num_heads, dropout):
    super().__init__()
    self.masked_attn_layer = MultiHeadAttention(d_model, num_heads)
    self.norm_attn_layer = MultiHeadAttention(d_model, num_heads)
    self.mlp = MLP(dff, d_model)

    self.dropout1 = tf.keras.layers.Dropout(dropout)
    self.dropout2 = tf.keras.layers.Dropout(dropout)

    self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-9)
    self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-9)
    self.layer_norm3 = tf.keras.layers.LayerNormalization(epsilon=1e-9)

  def call(self, dec_inputs, enc_outputs, padding_mask, look_ahead_mask):
    mask_attn_outputs, mask_attn_mat = self.masked_attn_layer(dec_inputs, dec_inputs, dec_inputs, look_ahead_mask)
    mask_attn_outputs = self.layer_norm1(dec_inputs + mask_attn_outputs)

    norm_attn_outputs, norm_attn_mat = self.norm_attn_layer(mask_attn_outputs, enc_outputs, enc_outputs, padding_mask)
    norm_attn_outputs = self.dropout1(norm_attn_outputs)
    norm_attn_outputs = self.layer_norm2(mask_attn_outputs + norm_attn_outputs)

    outputs = self.mlp(norm_attn_outputs)
    outputs = self.dropout2(outputs)
    outputs = self.layer_norm3(norm_attn_outputs + outputs)

    return outputs, mask_attn_mat, norm_attn_mat

In [11]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, num_layers, dff, d_model, num_heads, dropout):
    super().__init__()
    self.word_embedding = tf.keras.layers.Embedding(vocab_size, d_model)
    self.pos_embedding = PositionalEncoding(vocab_size, d_model)

    self.decoder_layer = [DecoderLayer(dff, d_model, num_heads, dropout) for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(dropout)

  def call(self, dec_inputs, enc_outputs, padding_mask, look_ahead_mask):
    mask_attn_mats, norm_attn_mats = [],  []
    embedding = self.word_embedding(dec_inputs)
    embedding = self.pos_embedding(embedding)
    outputs = self.dropout(embedding)

    for layer in self.decoder_layer:
      outputs, mask_attn_mat, norm_attn_mat  = layer(outputs, enc_outputs, padding_mask, look_ahead_mask)
      mask_attn_mats.append(mask_attn_mats)
      norm_attn_mats.append(norm_attn_mat)

    return outputs, mask_attn_mats, norm_attn_mats

In [12]:
class Transformer(tf.keras.Model):
  def __init__(self, enc_vocab_size, dec_vocab_size, num_layers, dff, d_model, num_heads, dropout):
    super().__init__()
    self.encoder = Encoder(enc_vocab_size, num_layers, dff, d_model, num_heads, dropout)
    self.decoder = Decoder(dec_vocab_size, num_layers, dff, d_model, num_heads, dropout)
    self.linear = tf.keras.layers.Dense(dec_vocab_size)

  def create_padding_mask(self, inputs):
    mask = tf.cast(tf.math.equal(inputs, 0), dtype=tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

  def create_look_ahead_mask(self, inputs):
    max_len = inputs.shape[1]
    #max_len = tf.shape(inputs)[1]
    mask = 1 - tf.linalg.band_part(tf.ones(shape=(max_len, max_len)), -1, 0)
    return mask

  def create_mask(self, enc_inputs, dec_inputs):
    enc_padding_mask = self.create_padding_mask(enc_inputs)

    dec_padding_mask = self.create_padding_mask(enc_inputs)

    look_ahead_mask = self.create_look_ahead_mask(dec_inputs)
    dec_look_padding_mask = self.create_padding_mask(dec_inputs)
    return enc_padding_mask, dec_padding_mask, tf.maximum(look_ahead_mask, dec_look_padding_mask)

  def call(self, inputs):
    enc_inputs, dec_inputs = inputs
    print(enc_inputs)
    enc_padding_mask, dec_padding_mask, look_ahead_mask = self.create_mask(enc_inputs, dec_inputs)
    
    enc_outputs, enc_attn_mats = self.encoder(enc_inputs, enc_padding_mask)
    dec_outputs, mask_attn_mats, dec_attn_mats = self.decoder(dec_inputs, enc_outputs, dec_padding_mask, look_ahead_mask)

    outputs = self.linear(dec_outputs)
    return outputs#, enc_attn_mats, mask_attn_mats, dec_attn_mats

# model parameter

In [13]:
ENC_VOCAB_SIZE = 12007
DEC_VOCAB_SIZE = 12007
NUM_LAYERS = 4
D_MODEL = 768
DFF = D_MODEL*4
NUM_HEADS = 8
DROPOUT = 0.1
MAX_LEN = 350

# data 로드

In [14]:
train_path_dir = "/content/drive/MyDrive/ColabNotebooks/project/translation/dataset/[라벨]ko2ja_culture_training_json.zip (Unzipped Files)"
val_path_dir = "/content/drive/MyDrive/ColabNotebooks/project/translation/dataset/[라벨]ko2ja_culture_validation_json.zip (Unzipped Files)"
train_file_list = os.listdir(train_path_dir)
val_file_list = os.listdir(val_path_dir)

In [15]:
train_data_df = pd.DataFrame()
for file in train_file_list:
  df = pd.read_json(path_or_buf=train_path_dir + '/' + file)
  train_data_df = pd.concat([train_data_df, df], ignore_index=False)

In [16]:
for file in val_file_list:
  df = pd.read_json(path_or_buf=val_path_dir + '/' + file)
  train_data_df = pd.concat([train_data_df, df], ignore_index=False)

In [17]:
ko_vocab_file = "/content/drive/MyDrive/ColabNotebooks/project/translation/vocab/kowiki.model"
ko_tokenizer = spm.SentencePieceProcessor()
ko_tokenizer.load(ko_vocab_file)

True

In [18]:
ja_vocab_file = "/content/drive/MyDrive/ColabNotebooks/project/translation/vocab/jpwiki.model"
ja_tokenizer = spm.SentencePieceProcessor()
ja_tokenizer.load(ja_vocab_file)

True

In [19]:
korean = []
japanese = []
for ko_sentence, ja_sentence in zip(train_data_df['한국어'], train_data_df['일본어']):
  korean.append(ko_sentence)
  japanese.append(ja_sentence)


In [20]:
del train_data_df

In [21]:
korean_pad_data = []

for line in korean:
  input = ko_tokenizer.encode_as_ids(line)
  input.insert(0, 2) #begin token
  input.append(3) #end token
  korean_pad_data.append(input)
korean_pad_data = tf.keras.preprocessing.sequence.pad_sequences(korean_pad_data, maxlen=MAX_LEN, padding='post')

In [22]:
del korean

In [23]:
japanese_pad_data = []

for line in japanese:
  input = ja_tokenizer.encode_as_ids(line)
  input.insert(0, 2)
  input.append(3)
  japanese_pad_data.append(input)
japanese_pad_data = tf.keras.preprocessing.sequence.pad_sequences(japanese_pad_data, maxlen=MAX_LEN, padding='post')

In [24]:
del japanese

In [25]:
print(korean_pad_data.shape)
print(japanese_pad_data.shape)

(1350000, 350)
(1350000, 350)


In [26]:
BATCH_SIZE_PER_REPLICA = 4
global_batch_size = (BATCH_SIZE_PER_REPLICA *
                     strategy.num_replicas_in_sync)

In [27]:
strategy.num_replicas_in_sync

8

In [28]:
dataset = tf.data.Dataset.from_tensor_slices((
    japanese_pad_data, korean_pad_data
))

dataset = dataset.cache()
#dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Training

In [29]:
batch_size=32

In [30]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LEN - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return loss

In [ ]:
with strategy.scope():
  model = create_model()
  optimizer = tf.keras.optimizers.Adam()
  training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)


@tf.function
def train_step(iterator):
  """The step function for one training step."""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    japanese, korean = inputs
    with tf.GradientTape() as tape:
      logits = model([japanese, korean[:, :-1]], training=True)
      loss = loss_function(korean[: 1:], logits)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)

  strategy.run(step_fn, args=(next(iterator),))

In [31]:
with strategy.scope():
  model = Transformer(ENC_VOCAB_SIZE, DEC_VOCAB_SIZE, NUM_LAYERS, DFF, D_MODEL, NUM_HEADS, DROPOUT)
  optimizer = tf.keras.optimizers.Adam()
  training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)

(1, 12007, 768)
(1, 12007, 768)


In [34]:
train_dataset = strategy.experimental_distribute_dataset(dataset)

In [36]:
def train_step(inputs):
  japanese, korean = inputs

  with tf.GradientTape() as tape:
    logits = model([japanese, korean[:, :-1]], training=True)
    loss = loss_function(korean[:, 1:], logits)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return loss 

@tf.function
def distributed_train_step(inputs):
  per_replica_losses = strategy.run(train_step, args=(inputs,))
  return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

for epoch in range(10):
  total_loss = 0.0
  num_batch = 0
  with tqdm(total=1350000//32 + 1, desc=f"Train({epoch})") as pbar:
    for x in train_dataset:
      total_loss += distributed_train_step(x)
      num_batch += 1

      pbar.update(1)
  train_loss = total_loss / num_batch

  model.save_weights('/content/drive/MyDrive/ColabNotebooks/project/translation/model/jp2ko_translation_model_weights.h5', overwrite=True)

In [ ]:
START_TOKEN, END_TOKEN = [ko_tokenizer.bos_id()], [ko_tokenizer.eos_id()]

# Evaluate and Predict

In [ ]:
def evaluate(sentence, model):
  if len(sentence) > MAXLEN:
    print("최대 길이 초과")
    return

  sentence = tf.expand_dims(
      START_TOKEN + ja_tokenizer.encode_as_ids(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAXLEN):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if np.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  output = tf.squeeze(output, axis=0)
  return output

In [ ]:
def predict(sentence, model):
  prediction = evaluate(sentence, model)
  return prediction

In [ ]:
load_model = Transformer(
      vocab_size=12007,
      num_layers=NUM_LAYERS,
      dff=DFF,
      d_model=D_MODEL,
      num_heads=NUM_HEADS,
      dropout=DROPOUT)

(1, 12007, 512)
(1, 12007, 512)


In [ ]:
load_model.load_weights('/content/drive/MyDrive/ColabNotebooks/project/translation/model/jp2ko_translation_model_weights.h5')

In [ ]:
def translate(sentences, load_model):
  sentence_list = sentences.split("。")
  sentence_list.pop()
  print(sentence_list)
  output_list = []
  for sentence in sentence_list:
    sentence += "。"
    output_list.append(predict(sentence, load_model))


  outputs = ""
  for output in output_list:
    new_output = []
    for word in output:
      new_output.append(int(word))
    
    output_de = ko_tokenizer.decode(new_output)
    outputs = outputs + '' + output_de
  return outputs

In [ ]:
sentences = "英政府は27日、国内でオミクロン株の感染を2件確認したと発表した。「オミクロン株は大変速いスピードで拡大している」。ジョンソン首相は記者会見で危機感をあらわにし、人口の約8割を占めるイングランドの店内や公共交通機関でマスク着用を義務づける新たな行動規制措置を発表した。また、すべての入国者に対して到着後2日以内のPCR検査や陰性が確認されるまでの自主隔離を義務づけるとも表明。オミクロン株の感染者と接触した人については10日間の自主隔離も必要とした。"

In [ ]:
output_list = translate(sentences, load_model)

In [ ]:
output_list

# 성능 확인
원문: 英政府は27日、国内でオミクロン株の感染を2件確認したと発表した。「オミクロン株は大変速いスピードで拡大している」。ジョンソン首相は記者会見で危機感をあらわにし、人口の約8割を占めるイングランドの店内や公共交通機関でマスク着用を義務づける新たな行動規制措置を発表した。また、すべての入国者に対して到着後2日以内のPCR検査や陰性が確認されるまでの自主隔離を義務づけるとも表明。オミクロン株の感染者と接触した人については10日間の自主隔離も必要とした。  
<br>
<br>
제작 모델: 영국 정부는 27일 국내에서 오미크론주 감염을 2건 확인했다고 밝혔다."오미크론주는 매우 빠른 속도로 확대되고 있다".존슨 총리는 기자회견에서 위기감을 드러내며 인구 약 8할을 차지하는 잉글랜드점 내나 대중교통에서 마스크 착용을 의무화하는 새로운 행동규제 조치를 발표했다.또 모든 입국자에 대해 도착 후 2일 이내 PCR 검사나 음성이 확인되기까지의 자가격리를 의무화한다고도 표명했다.오미크론주 확진자와 접촉한 사람에 대해서는 10일간의 자가격리도 필요로 했다.  
<br>
<br>
papago: 영국 정부는 27일, 국내에서 오미크론주의 감염을 2건 확인했다고 발표했다."오미크론주는 매우 빠른 속도로 확대하고 있다." 존슨 수상은 기자 회견에서 위기감을 나타내어, 인구의 약 8할을 차지하는 잉글랜드의 점내나 공공 교통 기관에서 마스크 착용을 의무화 하는 새로운 행동 규제 조치를 발표했다.또, 모든 입국자에 대해서 도착 후 2일 이내의 PCR 검사나 음성이 확인될 때까지의 자율 격리를 의무화한다고도 표명.오미크론주 감염자와 접촉한 사람에 대해서는 10일간의 자율격리도 필요로 했다.  